In [1]:
import numpy as np
import plotly.graph_objs as go
import plotly
from TX import euler

Euler angles:
$$  (\phi_1,\Phi,\phi_2) $$

$\phi_1$ rotates with respect to Z axis.



$a_{ij}=\mathbf{e}^\prime_{i}\cdot\mathbf{e}_{j}$

$$g^{(1)}=\begin{bmatrix} \cos\phi_1 & \cos(90^\circ-\phi_1) & 0 \\ \cos(90^\circ+\phi_1) & \cos\phi_1 & 0 \\ 0 & 0 & 1 \end{bmatrix}$$

$$g^{(2)}=\begin{bmatrix} 1 & 0 & 0 \\ 0 & \cos\Phi & \cos(90^\circ-\Phi) \\ 0 & \cos(90^\circ+\Phi) & \cos\Phi \end{bmatrix}$$

$$g^{(3)}=\begin{bmatrix} \cos\phi_2 & \cos(90^\circ-\phi_2) & 0 \\ \cos(90^\circ+\phi_2) & \cos\phi_2 & 0 \\ 0 & 0 & 1 \end{bmatrix}$$

$$g^{SA\gets CA}=g^{(3)}\cdot g^{(2)}\cdot g^{(1)}$$

인덱스 노테이션으로 표현하면 위는 아래와 같이 쓰인다.

$$g^{SA\gets CA}_{ij}=g^{(3)}_{ik} g^{(2)}_{kl} g^{(1)}_{lj}$$

보통, Euler angle을 활용해서 나타내는 방위 관계를 통해서 얻고자 하는 중요한 일은 '결정좌표계'에 나타난 물리량을 '시편좌표계'로 표현하는 것이다. 여러 단결정으로 이루어진 다결정의 '평균적'인 성질은, 각 단결정의 성질을 시편좌표계로 나타냈을 때의 평균값을 찾는 것이 목적이다. 따라서, ...

In [2]:
def calc_g(phi1,Phi,phi2):
    """
    Arguments
    ---------
    phi1, Phi, phi2
    """
    cos=np.cos
    ## degree to radian
    d90 =np.deg2rad(90.)    
    phi1=np.deg2rad(phi1)
    Phi =np.deg2rad(Phi)
    phi2=np.deg2rad(phi2)
    
    g1=np.identity(3)
    g2=np.identity(3)
    g3=np.identity(3)
    g =np.zeros((3,3))
    
    g1[0,0]=cos(phi1)
    g1[0,1]=cos(d90-phi1)
    g1[1,0]=cos(d90+phi1)
    g1[1,1]=cos(phi1)
    
    g2[1,1]=cos(Phi)
    g2[1,2]=cos(d90-Phi)
    g2[2,1]=cos(d90+Phi)
    g2[2,2]=cos(Phi)    
    
    g3[0,0]=cos(phi2)
    g3[0,1]=cos(d90-phi2)
    g3[1,0]=cos(d90+phi2)
    g3[1,1]=cos(phi2)
    

    for i in range(3):
        for j in range(3):
            for k in range(3):
                for l in range(3):
                    g[i,j]=g[i,j]+g3[i,k]*g2[k,l]*g1[l,j]
    return g1,g2,g3,g

In [3]:
phi1,phi,phi2=-30,40,0
g1,g2,g3,g=calc_g(phi1,phi,phi2)
print(g)
#g=euler.euler(phi1,phi,phi2,echo=False)
#print(g)

[[ 8.66025404e-01 -5.00000000e-01  3.93593894e-17]
 [ 3.83022222e-01  6.63413948e-01  6.42787610e-01]
 [-3.21393805e-01 -5.56670399e-01  7.66044443e-01]]


In [8]:
## unit cell
def draw_unit(fig,m2v,cdim):
    xys=np.array([[0,0,0],[0,1,0],[1,1,0],[1,0,0],[0,0,1],[0,1,1],[1,1,1],[1,0,1]],dtype='float')
    for i in range(3):
        xys[:,i]=xys[:,i]*cdim[i]
    orig=np.array(cdim)/2.#np.array([cdim[0]/2,0.5,0.5]) ## center of unit cell
    xys=xys-orig

    xys_init=xys.copy()
    for i in range(len(xys)):
        v=xys[i,:]
        newv=np.dot(v,m2v)
        xys[i,:]=newv

    xys     =xys.T
    xys_init=xys_init.T
    xys_all=[xys,xys_init]

    #the start and end point for each line
    pairs = [[0,1],[1,2],[2,3],[3,0],[4,5],[5,6],[6,7],[7,4],[0,4],[1,5],[2,6],[3,7]]
    xys_lines=[
        [[],[],[]], # initial_unit_cell
        [[],[],[]],  # rotated_unit_cell
    ]

    labels=['initial unit cell','rotated unit cell']
    cs=['black','green']
    modes=['line','line']

    #create the coordinate list for the lines
    for p in pairs:
        iline=0
        for k in range(2): ## two lines
            for i in range(2): ## start and ends
                for j in range(3): ## xys coordinates
                    xys_lines[iline][j].append(xys_all[iline][j][p[i]])
            ## breaks
            for j in range(3):
                xys_lines[iline][j].append(None)
            iline=iline+1


    #fig = go.Figure()
    for i in range(len(xys_lines)):
        x,y,z=xys_lines[i]
        trace=go.Scatter3d(x=x,y=y,z=z,mode='lines',
                           name=labels[i],line={'color':cs[i]})
        fig.add_trace(trace)

In [9]:
## create plotly figure

fig=go.Figure()

## Euler angles
phi1=-20.
phi = 50.
phi2= 80.

## dimension and angle of unit cell.
cdim=[0.2,0.2,0.5]
cang=[90,90,90]

## Calculate orientation
g1,g2,g3,g=calc_g(phi1,phi,phi2)
m2v=g.T

## draw unit cell
draw_unit(fig,m2v=m2v,cdim=cdim)
fig.show()